In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import re

In [2]:
main_link = 'https://hh.ru'

search_vacancy = 'data scientist'
page = 0
while True:
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

    params = {'text': search_vacancy,
              'clusters': 'true',
              'enable_snippets': 'true',
              'salary': '',
              'st': 'searchVacancy',
              'fromSearch': 'true',
              'page': page}

    response = requests.get(main_link + '/search/vacancy/', params=params, headers=header)
    soup = bs(response.text, 'html.parser')

    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})

    vacancy_list = vacancy_block.findChildren(recursive=False)

    vacancies = []

    for vacancy in vacancy_list:
        if vacancy['class'] == ['vacancy-serp-item'] or vacancy['class'] == ['vacancy-serp-item', 'vacancy-serp-item_premium']:
            link = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
            company = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).text
            city = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).text
            name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
            salary = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text
            if len(salary) == 0:
                min_salary, max_salary = float('nan'), float('nan')
                currency = [float('nan')]
            else:
                currency = re.search('([^\d]*$)', salary)
                _sal = salary.split('-')
                if len(_sal) == 1:
                    if 'от' in _sal[0]:
                        min_salary = re.sub('\D', '', _sal[0])
                        max_salary = float('nan')
                    else:
                        max_salary = re.sub('\D', '', _sal[0])
                        min_salary = float('nan')
                else:
                    min_salary = re.sub('\D', '', _sal[0])
                    max_salary = re.sub('\D', '', _sal[1])
#             try:
#                 salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
#             except AttributeError:
#                 salary = 'None'
#             salary = salary.replace('\xa0', '')

            vacancy_data = {'link': link,
                            'company': company,
                            'city': city,
                            'name': name,
                            'min_salary': min_salary,
                            'max_salary': max_salary,
                            'cur': currency[0]}

            vacancies.append(vacancy_data)

    page += 1

    if soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}) is None:
        break


In [3]:
df_vacancies = pd.DataFrame(vacancies)

In [4]:
df_vacancies.head(15)

,link,company,city,name,min_salary,max_salary,cur
0,https://hh.ru/vacancy/36859214?query=data%20sc...,Grid Dynamics,Саратов,Senior Data Scientist,NaN,NaN,NaN
1,https://hh.ru/vacancy/36694503?query=data%20sc...,"ПАО МегаФон, Офис","Москва, Чеховская и еще 2",Senior data scientist,NaN,NaN,NaN
2,https://hh.ru/vacancy/36996831?query=data%20sc...,ООО Гарда Технологии,"Нижний Новгород, Горьковская",Аналитик данных (Data analyst/Data Scientist),NaN,NaN,NaN
3,https://hh.ru/vacancy/37008699?query=data%20sc...,"АБСОЛЮТ, Группа",Москва,Продуктовый и маркетинговый аналитик,NaN,NaN,NaN
4,https://hh.ru/vacancy/37000491?query=data%20sc...,True Engineering,Новосибирск,Senior Data Scientist / Machine Learning Devel...,NaN,NaN,NaN
5,https://hh.ru/vacancy/36691192?query=data%20sc...,"ПАО МегаФон, IT","Москва, Чеховская и еще 2",Team Lead data scientist (коммерческое направл...,NaN,NaN,NaN
6,https://hh.ru/vacancy/37011667?query=data%20sc...,Bell Integrator,Москва,Technical Director of SaaS product,NaN,NaN,NaN
7,https://hh.ru/vacancy/37080498?query=data%20sc...,ООО Стандарт Стафф,Москва,Frontend Vue Developer,NaN,NaN,NaN
8,https://hh.ru/vacancy/37125980?query=data%20sc...,SegmentStream,Москва,Marketing Manager (global market),NaN,NaN,NaN
9,https://hh.ru/vacancy/37004358?query=data%20sc...,ЕвразХолдинг,Новосибирск,Python+ ML&DL (Data Scientist),NaN,NaN,NaN
